In [3]:
import torch

In [159]:
W1 = torch.randn(4, 3).requires_grad_(True)
W2 = torch.randn(4, 4).requires_grad_(True)
W3 = torch.randn(1, 4).requires_grad_(True)
def sdf(x):
    return W3 @ torch.relu(W2 @ torch.relu(W1 @ x))

x = torch.randn(3,1).requires_grad_(True)
print(x)
s = sdf(x)
print(s)

# torchviz.make_dot(s, params={'W0' : W0, 'W1' : W1, 'pos': x, 'sdf': s})

tensor([[ 2.0329],
        [-2.0475],
        [-1.4333]], requires_grad=True)
tensor([[-5.4081]], grad_fn=<MmBackward0>)


In [160]:
dsdf_dpos, = torch.autograd.grad([s], [x], retain_graph=True, create_graph=True)
print(dsdf_dpos)

dsdf_dpos_formula = ( W3*(W2@(torch.relu(W1@x))>=0).T ) @ (W2*(W1@x>=0).T) @ W1

print(dsdf_dpos_formula)

tensor([[-0.7697],
        [ 1.6181],
        [ 0.3701]], grad_fn=<MmBackward0>)
tensor([[-0.7697,  1.6181,  0.3701]], grad_fn=<MmBackward0>)


In [161]:
dsdf_dpos.retain_grad()

L_eikonal = (torch.sqrt(dsdf_dpos[0]**2 + dsdf_dpos[1]**2 + dsdf_dpos[2]**2) - 1)**2
L_eikonal.backward(retain_graph=True)

print(dsdf_dpos.grad)
print(W1.grad)
print(W2.grad)
print(W3.grad)

# torchviz.make_dot(L_eikonal, params={'W0' : W0, 'W1' : W1, 'pos': x, 'sdf': s, 'L_sdf': L_sdf})

tensor([[-0.6980],
        [ 1.4674],
        [ 0.3356]])
tensor([[ 0.0000,  0.0000,  0.0000],
        [ 0.4720, -0.9923, -0.2270],
        [ 0.2914, -0.6126, -0.1401],
        [ 0.0000,  0.0000,  0.0000]])
tensor([[0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 1.0122, 1.9597, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000]])
tensor([[ 0.0000,  0.0000, -6.1332,  0.0000]])


In [111]:
a = torch.sqrt(dsdf_dpos[0]**2 + dsdf_dpos[1]**2 + dsdf_dpos[2]**2)
dL_dxyz = 2 * (a - 1) / a * dsdf_dpos
dL_dxyz

tensor([[ 1.0038],
        [ 0.0351],
        [-0.0443]], grad_fn=<MulBackward0>)

In [88]:
W2*(W1 @ x>=0).T

tensor([[-0.0000,  0.0000, -0.9765, -0.0000],
        [-0.0000,  0.0000,  0.1501,  0.0000],
        [-0.0000,  0.0000,  1.1988,  0.0000],
        [ 0.0000,  0.0000,  1.1419, -0.0000]], grad_fn=<MulBackward0>)

In [106]:
dL_dW3=W2@W1@dL_dxyz

print(dL_dW3)
(W1@x>=0)

tensor([[ 1.9835],
        [ 1.9141],
        [-1.5980],
        [-4.7289]], grad_fn=<MmBackward0>)


tensor([[False],
        [False],
        [ True],
        [False]])

In [85]:
(W2@(torch.relu(W1@x))>=0)

tensor([[False],
        [ True],
        [ True],
        [ True]])

In [6]:
dL_dW0 = dL_dxyz.unsqueeze(1) @ W1.T * (W0.T @ x >= 0).float()
dL_dW0

tensor([[ 0.2041, -0.0960, -0.0000,  0.0672],
        [-0.5144,  0.2419,  0.0000, -0.1694],
        [-0.3787,  0.1781,  0.0000, -0.1247]], grad_fn=<MulBackward0>)

In [15]:
t = (W0.T @ x).float().sigmoid()
dL_dW1 = (dL_dxyz.unsqueeze(1) * W0 * t*(1-t)).sum(0)
dL_dW1

tensor([ 0.2481, -0.1413,  0.1050,  0.2448], grad_fn=<SumBackward1>)